In [1]:
import numpy as np
from numpy.linalg import norm
import scipy
import time 
import pandas as pd
from scipy.optimize import nnls
import matplotlib.pyplot as plt
import scipy.sparse.linalg

# Basic functions

In [ ]:
def mus(n,a=-1,b=1):
    mu = lambda a,b,k:(b**(k+1)-a**(k+1))/(k+1)
    res = [mu(a,b,k) for k in range(n)]
    return np.array(res)

def x_matrix(K,xs):
    return np.array([xs**k for k in range(K)])

def wxks(n,xs,ws):

    res = np.zeros(n)
    for k in range(n):
        res[k] = (xs**k)@ws
    return res

def f(point,a=-1,b=1):
    n = point.shape[0]

    xs = point[:n//2]
    ws = point[n//2:]
    
    Mus = mus(n,a,b)
    Wxks = wxks(n,xs,ws)
    return ((Mus-Wxks)**2).sum()

def grad_x(point,a=-1,b=1):
    n = point.shape[0]
    xs = point[:n//2]
    ws = point[n//2:]
    result = np.zeros(n)
    Ms = mus(n,a,b)
    Wxks = wxks(n,xs,ws) 
    for j in range(n):
        for k in range(n):
            term1 = Ms[k]
            term2 = (xs**k)@ws 
            if j <= n/2 - 1:
                if k == 0:
                    result[j] += 0
                else:
                    result[j] += -2*ws[j]*k*(xs[j])**(k-1)*(term1-term2)
            else:
                result[j] += 0
    return result

def grad_w(point,a=-1,b=1):
    n = point.shape[0]
    xs = point[:n//2]
    ws = point[n//2:]
    result = np.zeros(n)
    Ms = mus(n,a,b)
    Wxks = wxks(n,xs,ws) 
    for j in range(n):
        for k in range(n):
            term1 = Ms[k]
            term2 = (xs**k)@ws 
            if j <= n/2 - 1:
                result[j] += 0
            else:
                result[j] += -2*((xs[j-n//2])**k)*(term1-term2)
    return result   

def grad_f(point,a=-1,b=1):
    n = point.shape[0]
    xs = point[:n//2]
    ws = point[n//2:]
    result = np.zeros(n)
    Ms = mus(n,a,b)
    Wxks = wxks(n,xs,ws) 
    for j in range(n):
        for k in range(n):
            term1 = Ms[k]
            term2 = (xs**k)@ws 
            if j <= n/2 - 1:
                if k == 0:
                    result[j] += 0
                else:
                    result[j] += -2*ws[j]*k*(xs[j])**(k-1)*(term1-term2)
            else:
                result[j] += -2*((xs[j-n//2])**k)*(term1-term2)
    return result

# Projection matrix of active set

In [ ]:
def epsilon_inactive(x,epsilon,a=-1,b=1):
    n = x.shape[0]
    xs , ws = x[:n//2] , x[n//2:]
    term1 = ((xs<b-epsilon) & (xs>a+epsilon))
    term2 = (ws>epsilon) 
    return np.diag(np.concatenate([term1,term2]))

def epsilon_active(x,epsilon,a=-1,b=1):
    n = x.shape[0]
    xs , ws = x[:n//2] , x[n//2:]
    term1 = ((xs>=b-epsilon) | (xs<=a+epsilon))
    term2 = (ws<=epsilon)
    
    return np.diag(np.concatenate([term1,term2]))

# Algorithms in 5.5.3

In [ ]:
def project(point,a=-1,b=1):
    n = point.shape[0]
    xs = point[:n//2]
    ws = point[n//2:]
    xs = np.where(xs>=a,xs,a)
    xs = np.where(xs<=b,xs,b)
    ws = np.where(ws>=0,ws,0)
    return np.concatenate([xs,ws],axis=0)

def project_line_search(point,p):
    alpha = 1
    pj_point = project(point + alpha*p)
    while f(pj_point) - f(point) > -1e-4/alpha*(norm(pj_point-point))**2:
        alpha *= 0.5
        if alpha <= 1e-100:
            break
        pj_point = project(point + alpha*p)
    return alpha

def bfgsrec(n,sks,yks,H0,d):
    ## Base case
    if n == 0:
        return H0@d
    rhok_inv = (yks[n-1]).T@sks[n-1]

    ## Avoid overflow
    if rhok_inv == 0 :
            rhok = 1. / rhok_inv 
    else:
        rhok = 1. / rhok_inv 

    alpha = (sks[n-1]).T@d/rhok
    d -= alpha*yks[n-1]
    d = bfgsrec(n-1,sks,yks,H0,d)
    d += (alpha - ((yks[n-1]).T@d/rhok))*sks[n-1]
    return d

def bfgsrecb(n,sk3,yk3,A0,d,pI):
    ## Base case
    d = pI@d
    if n==0:
        return A0@d
    rhok_inv = (yk3[n-1]).T@sk3[n-1]


    ## Avoid overflow
    if rhok_inv == 0 :
            rhok = 1. / rhok_inv 
    else:
        rhok = 1. / rhok_inv 
    alpha = (sk3[n-1]).T@d/rhok
    d -= alpha*yk3[n-1]
    d = bfgsrec(n-1,sk3,yk3,A0,d)
    d += (alpha - (yk3[n-1].T@d/rhok)*sk3[n-1])*sk3[n-1]
    return pI@d

def bfgsoptb(x,tau_a=1,tau_r=1,grad=grad_f):
    ns,n=0,0
    pg0 = x-project(x-grad(x))
    pg = pg0
    epsilon = min(1,norm(pg))
    pA = epsilon_active(x,epsilon)
    pI = epsilon_inactive(x,epsilon)
    A0 = pI
    sks,yks = [0]*1000,[0]*1000
    F = []
    while norm(pg) <= tau_a+tau_r*norm(pg0):
        d = -grad(x)
        d = bfgsrecb(ns,sks,yks,np.linalg.pinv(A0),d,pI)
        d -= pA@grad(x)
        lam = project_line_search(x,d)
        sks[ns] = pI@(project(x+lam*d)-x)

        xp = project(x+lam*d)
        y = grad(xp)-grad(x)
        x=xp

        yks[ns] = pI@y

        ## Check if the update should be skipped
        if yks[ns].T@sks[ns]>0:
            ns += 1
        else:
            ns = 0
        pg = x-project(x-grad(x))
        epsilon = min(1,norm(pg))
        pA = epsilon_active(x,epsilon)
        pI = epsilon_inactive(x,epsilon)
        n += 1
        F.append(f(x))

        ## Check for termination
        if n >= 1000:
            break
    return x,F
    

# Use bfgsoptb in an alternative manner

In [ ]:
def alternative_ls(xs):
    n = xs.shape[0]
    xm = x_matrix(2*n,xs)
    it = 0
    while True:
        it += 1
        ws = nnls(xm,mus(2*n))[0]
        point = np.concatenate([xs,ws])
        point_proposed = bfgsoptb(point,grad=grad_x)[0]
        if f(point_proposed) <= 1e-15:
            break
        
        xs = point_proposed[:n]
        xm = x_matrix(2*n,xs)

    return point[:n] , point[n:]

# My own implementation of projected bfgs (without using recursion)

In [674]:
def bfgs(x0, grad=grad_f,maxiter=None):



    if maxiter is None:
        maxiter = 1000


    gfk = grad(x0)

    k = 0
    N = len(x0)
    I = np.eye(N, dtype=int)

    r_0 = norm(x0 - project(x0 - grad(x0)))
    t_a = 1e-12
    t_r = 1e-12
    
    pg0 = x0 - project(x0 - grad(x0))
    pg = pg0
    xk = x0
    Hk = I
    pI=I
    residual = f(xk)
    while k < maxiter:
        # pk = scipy.sparse.linalg.cg(Hk, gfk, tol=min(1e-5, residual))[0]
        pk = -(Hk@gfk)
        alpha_k = project_line_search(xk,pk)
        xkp1 = project(xk + alpha_k * pk)
        
        pg = xkp1 - project(xkp1-grad(xkp1))
        pI_ = epsilon_inactive(xkp1,min(1,norm(pg)))

        sk = pI_@(xkp1 - xk)
        xk = xkp1
        residual = f(xk)
        gfkp1 = grad(xkp1)

        yk = pI_@(gfkp1 - gfk)
        if yk.T@sk <0:
            Hk = pI
        gfk = gfkp1

        k += 1
        
        if (norm(f(xk)) <= t_a + t_r*r_0):
            print('Converges at {}th iteration.'.format(k))
            break


        rhok_inv = np.dot(yk, sk)
        # Avoid illegal divide
        if rhok_inv == 0.:
            rhok = 1000.0
        else:
            rhok = 1. / rhok_inv
        # Hk = pI@Hk@pI + yk[:, np.newaxis] * yk[np.newaxis, :] * rhok - pI@((Hk@sk)[:, np.newaxis]) * ((Hk@sk)[np.newaxis, :])@pI * rhok
        A1 = I - sk[:, np.newaxis] * yk[np.newaxis, :] * rhok
        A2 = I - yk[:, np.newaxis] * sk[np.newaxis, :] * rhok

        ## Check if our active set is the same
        if norm(pI-pI_) > 0:
            Hk = np.dot(A1, np.dot(pI@Hk@pI, A2)) + (rhok * sk[:, np.newaxis] *
                                                 sk[np.newaxis, :])
        else:
            Hk = np.dot(A1, np.dot(Hk, A2)) + (rhok * sk[:, np.newaxis] *
                                                 sk[np.newaxis, :])
    # if k == maxiter:
        # print('After {} iters, we get a gradient with norm {}.'.format(k,norm(pg)))
    
    return xk

# Using bfgs to optimize (xs,ws) as a whole

In [676]:
n = 99
xs = np.linspace(-1,1,n)
ws = np.ones(n)
point = np.concatenate([xs,ws])
res_bfgs = bfgs(point)
f(res_bfgs),res_bfgs

(0.92428753514279,
 array([ 8.84841946e-02,  8.84841946e-02,  8.84841946e-02,  8.84841946e-02,
         8.84841946e-02,  8.84841946e-02,  8.84841946e-02,  8.84841946e-02,
         8.84841946e-02,  8.84841946e-02,  8.84841946e-02,  8.84841946e-02,
         8.84841946e-02,  8.84841946e-02,  8.84841946e-02,  8.84841946e-02,
         8.84841946e-02,  8.84841946e-02,  8.84841946e-02,  8.84841946e-02,
         8.84841946e-02,  8.84841946e-02,  8.84841946e-02,  8.84841946e-02,
         8.84841946e-02,  8.84841946e-02,  8.84841946e-02,  8.84841946e-02,
         8.84841946e-02,  8.84841946e-02,  8.84841946e-02,  8.84841946e-02,
         8.84841946e-02,  8.84841946e-02,  8.84841946e-02,  8.84841946e-02,
         8.84841946e-02,  8.84841946e-02,  8.84841946e-02,  8.84841946e-02,
         8.84841946e-02,  8.84841946e-02,  8.84841946e-02,  8.84841946e-02,
         8.84841946e-02,  8.84841946e-02,  8.84841946e-02,  8.84841946e-02,
         8.84841946e-02,  1.05411286e-11, -8.84841946e-02, -8.8484194

# Using iterative bfgs in an alternative manner

In [ ]:
def alternative_ls_(xs):
    n = xs.shape[0]
    xm = x_matrix(2*n,xs)
    it = 0
    while True:
        it += 1
        ws = nnls(xm,mus(2*n))[0]
        point = np.concatenate([xs,ws])
        point_proposed = bfgs(point,grad=grad_x)
        if f(point_proposed) <= 1e-15:
            break
        if it >= 100:
            break
        xs = point_proposed[:n]
        xm = x_matrix(2*n,xs)

    return point[:n] , point[n:]

In [ ]:
n = 5
xs = np.linspace(-1,1,n)
ws = np.ones(n)
t1 = time.time()
res = alternative_ls_(xs)
print(time.time()-t1)
print(res)
